In [1]:
import pickle
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame

In [22]:
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [3]:
def unpickle(file):
    with open(file,'rb') as f:
        dict = pickle.load(f,encoding='latin1')
    return dict
def one_hot(x, n):
    x = np.array(x)
    assert x.ndim == 1
    return np.eye(n)[x]

In [4]:
datadir="./dataSet/cifar_10"

In [5]:
def get_dat_set(name):
    x = []
    y = []
    if name == "train":
        for i in range(5):
            datadict = unpickle(datadir+"/data_batch_"+str(i+1))
            X = datadict["data"]
            Y = datadict["labels"]
            X = np.array(X,dtype=float)/255.0
            X = X.reshape([-1,3,32,32]).transpose([0,2,3,1])
            x.extend(X)
            y.extend(Y)
        x = np.array(x)
        y = np.array(y)
    elif name =="test":
        datadict = unpickle(datadir+"/test_batch")
        x = datadict["data"]
        y = np.array(datadict['labels'])
        x = np.array(x, dtype=float) / 255.0
        x = x.reshape([-1, 3, 32, 32])
        x = x.transpose([0,2,3,1])   
    return x, one_hot(y,10)

In [25]:
train_data,train_label = get_dat_set(name="train")
test_data, test_label = get_dat_set(name="test")

In [26]:
print("Train_data: {}".format(train_data.shape))

Train_data: (50000, 32, 32, 3)


In [27]:
train_label.shape

(50000, 10)

In [28]:
# mobile_model = keras.applications.mobilenet.MobileNet()

# model = Sequential()
# for layer in mobile_model.layers[:-3]:
#     layer.trainable = True
#     model.add(layer)

# model.add(Dense(10,activation='softmax'))
# model.summary()

In [30]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=train_data.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('softmax'))

In [36]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_6 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_7 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 15, 15, 64)        18496     
__________

In [31]:
model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [33]:
model.fit(x = train_data,y=train_label,batch_size=50,epochs=50,verbose=2)

Epoch 1/50
 - 187s - loss: 1.5180 - acc: 0.4436
Epoch 2/50
 - 191s - loss: 1.1446 - acc: 0.5938
Epoch 3/50
 - 197s - loss: 0.9851 - acc: 0.6524
Epoch 4/50
 - 206s - loss: 0.8789 - acc: 0.6932
Epoch 5/50
 - 202s - loss: 0.8077 - acc: 0.7185
Epoch 6/50
 - 203s - loss: 0.7544 - acc: 0.7369
Epoch 7/50
 - 204s - loss: 0.7127 - acc: 0.7518
Epoch 8/50
 - 205s - loss: 0.6785 - acc: 0.7622
Epoch 9/50
 - 211s - loss: 0.6454 - acc: 0.7726
Epoch 10/50
 - 204s - loss: 0.6128 - acc: 0.7830
Epoch 11/50
 - 296s - loss: 0.5974 - acc: 0.7905
Epoch 12/50
 - 205s - loss: 0.5721 - acc: 0.7993
Epoch 13/50
 - 208s - loss: 0.5502 - acc: 0.8061
Epoch 14/50
 - 206s - loss: 0.5369 - acc: 0.8098
Epoch 15/50
 - 206s - loss: 0.5195 - acc: 0.8157
Epoch 16/50
 - 208s - loss: 0.5038 - acc: 0.8219
Epoch 17/50
 - 208s - loss: 0.4896 - acc: 0.8255
Epoch 18/50
 - 208s - loss: 0.4780 - acc: 0.8300
Epoch 19/50
 - 261s - loss: 0.4676 - acc: 0.8353
Epoch 20/50
 - 199s - loss: 0.4558 - acc: 0.8390
Epoch 21/50
 - 209s - loss: 0

In [37]:
scores = model.evaluate(test_data, test_label,verbose=2)

[0.6611566251039505, 0.7969]

In [39]:
print("Test loss: {:.4f}".format(scores[0]))
print("Test accuracy: {:.4f}".format(scores[1]))

Test loss: 0.6612
Test accuracy: 0.7969
